# Tests techniques - Affluences 

In [50]:
import pandas as pd

In [51]:
data = pd.read_csv("data/data.csv")
data.head()

,sensor_identifier,sensor_name,site_id,last_record_datetime
0,3,sensor_3,388,2019-01-30 13:55:00
1,4,sensor_4,389,2019-01-30 14:00:00
2,7,sensor_7,1784,2021-03-25 06:11:00
3,8,sensor_8,1784,2021-05-05 15:17:00
4,9,sensor_9,443,2021-05-05 15:15:00


In [52]:
tt = pd.read_csv("data/timetables.csv")
tt.head()

,site_id,opening_datetime,closing_datetime
0,107,2021-05-06 10:00:00,2021-05-06 18:00:00
1,132,2021-05-06 07:00:00,2021-05-06 19:00:00
2,233,2021-05-06 00:00:00,2021-05-06 23:55:00
3,235,2021-05-06 00:00:00,2021-05-06 23:55:00
4,272,2021-05-06 08:30:00,2021-05-06 18:00:00


In [53]:
df = pd.merge(left=data, right=tt, left_on='site_id', right_on='site_id')
df.head()

,sensor_identifier,sensor_name,site_id,last_record_datetime,opening_datetime,closing_datetime
0,3,sensor_3,388,2019-01-30 13:55:00,2021-05-06 00:00:00,2021-05-06 23:55:00
1,4,sensor_4,389,2019-01-30 14:00:00,2021-05-06 11:00:00,2021-05-06 18:00:00
2,7,sensor_7,1784,2021-03-25 06:11:00,NaN,NaN
3,8,sensor_8,1784,2021-05-05 15:17:00,NaN,NaN
4,9,sensor_9,443,2021-05-05 15:15:00,2021-05-06 00:00:00,2021-05-06 23:55:00


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209 entries, 0 to 208
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   sensor_identifier     209 non-null    int64 
 1   sensor_name           209 non-null    object
 2   site_id               209 non-null    int64 
 3   last_record_datetime  209 non-null    object
 4   opening_datetime      183 non-null    object
 5   closing_datetime      183 non-null    object
dtypes: int64(2), object(4)
memory usage: 11.4+ KB


In [55]:
df["last_record_datetime"] = pd.to_datetime(df["last_record_datetime"])
df["opening_datetime"] = pd.to_datetime(df["opening_datetime"], errors="coerce")
df["closing_datetime"] = pd.to_datetime(df["closing_datetime"], errors="coerce")
df.head()

,sensor_identifier,sensor_name,site_id,last_record_datetime,opening_datetime,closing_datetime
0,3,sensor_3,388,2019-01-30 13:55:00,2021-05-06 00:00:00,2021-05-06 23:55:00
1,4,sensor_4,389,2019-01-30 14:00:00,2021-05-06 11:00:00,2021-05-06 18:00:00
2,7,sensor_7,1784,2021-03-25 06:11:00,NaT,NaT
3,8,sensor_8,1784,2021-05-05 15:17:00,NaT,NaT
4,9,sensor_9,443,2021-05-05 15:15:00,2021-05-06 00:00:00,2021-05-06 23:55:00


In [56]:
df["datetime_now"] = pd.to_datetime('2021-05-06 15:00:00')
df["diff"] = (df["datetime_now"]- df["last_record_datetime"]) / pd.Timedelta(hours=1)
df.head()

,sensor_identifier,sensor_name,site_id,last_record_datetime,opening_datetime,closing_datetime,datetime_now,diff
0,3,sensor_3,388,2019-01-30 13:55:00,2021-05-06 00:00:00,2021-05-06 23:55:00,2021-05-06 15:00:00,19849.083333
1,4,sensor_4,389,2019-01-30 14:00:00,2021-05-06 11:00:00,2021-05-06 18:00:00,2021-05-06 15:00:00,19849.000000
2,7,sensor_7,1784,2021-03-25 06:11:00,NaT,NaT,2021-05-06 15:00:00,1016.816667
3,8,sensor_8,1784,2021-05-05 15:17:00,NaT,NaT,2021-05-06 15:00:00,23.716667
4,9,sensor_9,443,2021-05-05 15:15:00,2021-05-06 00:00:00,2021-05-06 23:55:00,2021-05-06 15:00:00,23.750000


In [57]:
opened = df[((df["opening_datetime"] <= df["datetime_now"]) & (df["closing_datetime"] >= df["datetime_now"]))]
alert = opened[opened["diff"] > 2].copy()
alert["level"] = alert["diff"].apply((lambda x: 3 if x > 48 else (2 if x > 24 else 1)))

cleaned = alert[["sensor_identifier", "sensor_name", "datetime_now", "last_record_datetime", "level"]]
cleaned.head()

,sensor_identifier,sensor_name,datetime_now,last_record_datetime,level
0,3,sensor_3,2021-05-06 15:00:00,2019-01-30 13:55:00,3
1,4,sensor_4,2021-05-06 15:00:00,2019-01-30 14:00:00,3
4,9,sensor_9,2021-05-06 15:00:00,2021-05-05 15:15:00,1
5,10,sensor_10,2021-05-06 15:00:00,2021-05-05 15:15:00,1
6,142,sensor_142,2021-05-06 15:00:00,2021-05-05 15:15:00,1


In [58]:
cleaned.head()

,sensor_identifier,sensor_name,datetime_now,last_record_datetime,level
0,3,sensor_3,2021-05-06 15:00:00,2019-01-30 13:55:00,3
1,4,sensor_4,2021-05-06 15:00:00,2019-01-30 14:00:00,3
4,9,sensor_9,2021-05-06 15:00:00,2021-05-05 15:15:00,1
5,10,sensor_10,2021-05-06 15:00:00,2021-05-05 15:15:00,1
6,142,sensor_142,2021-05-06 15:00:00,2021-05-05 15:15:00,1


In [59]:
for _, r in cleaned.iterrows():
    print(f"Sensor {r['sensor_name']} with identifier {r['sensor_identifier']} triggers an alert at {r['datetime_now']} with level {r['level']} with last data recorded at {r['last_record_datetime']}")
    print("")

Sensor sensor_3 with identifier 3 triggers an alert at 2021-05-06 15:00:00 with level 3 with last data recorded at 2019-01-30 13:55:00

Sensor sensor_4 with identifier 4 triggers an alert at 2021-05-06 15:00:00 with level 3 with last data recorded at 2019-01-30 14:00:00

Sensor sensor_9 with identifier 9 triggers an alert at 2021-05-06 15:00:00 with level 1 with last data recorded at 2021-05-05 15:15:00

Sensor sensor_10 with identifier 10 triggers an alert at 2021-05-06 15:00:00 with level 1 with last data recorded at 2021-05-05 15:15:00

Sensor sensor_142 with identifier 142 triggers an alert at 2021-05-06 15:00:00 with level 1 with last data recorded at 2021-05-05 15:15:00

Sensor sensor_11 with identifier 11 triggers an alert at 2021-05-06 15:00:00 with level 1 with last data recorded at 2021-05-05 15:15:00

Sensor sensor_11 with identifier 11 triggers an alert at 2021-05-06 15:00:00 with level 1 with last data recorded at 2021-05-05 15:15:00

Sensor sensor_13 with identifier 13 tr

In [60]:
# Check only
#alert["opening_datetime"].isnull().sum())
assert alert["opening_datetime"].isnull().sum() == 0
assert alert["closing_datetime"].isnull().sum() == 0

In [61]:
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109 entries, 0 to 205
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   sensor_identifier     109 non-null    int64         
 1   sensor_name           109 non-null    object        
 2   datetime_now          109 non-null    datetime64[ns]
 3   last_record_datetime  109 non-null    datetime64[ns]
 4   level                 109 non-null    int64         
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 5.1+ KB
